# Calculations of $g^{(2)}(\omega_1,\omega_2)$ for a squeezed laser

In [50]:
from qutip import *
import numpy as np
from numpy import sqrt
import time
import matplotlib.pyplot as plt

from functions import *


Definition of parameters

In [51]:
g = 1
ratio = 0.01
C = 1.
P = sqrt(4/(C*ratio))
kappa = ratio*P
gsens = 0.0001
Gamma = 0.01

# Squeezing parameter
r = 0.4

# Truncation parameter
Ncav = 60

In [52]:
DeltaList = [-0.1,0.1]
g2g1(DeltaList,g,gsens,kappa,P,Gamma,r,Ncav)

TypeError: g2g1() takes from 6 to 7 positional arguments but 8 were given

In [49]:
(DeltaList,g,gsens,kappa,P,Gamma,r,Ncav)

([-0.1, 0.1], 1, 0.0001, 0.2, 20.0, 0.01, 0.4, 60)